In [1]:
# Your original list
numbers = [110224, 146930, 2941, 104545, 62327, 29760, 96891, 47026, 117733, 163207, 
           61451, 20590, 145423, 33883, 4524, 81255, 82144, 85139, 167094, 125904, 
           116418, 158098, 95233, 81836, 84071, 53850, 112264, 17238, 34056, 10708, 
           164819, 32413, 75065, 139894, 161233, 100688, 148030, 55853, 23077, 152167, 
           44378, 111683, 145091, 132167, 83409, 153865, 143858, 111283, 150657, 36240, 
           142480, 112754, 149469, 50328, 163801, 45304, 155756, 151190, 160715, 8876, 
           116827, 142384, 143574, 136796, 278, 120989, 120991, 151642, 44083, 94917, 
           32815, 117614, 98217, 41937, 16626, 163885, 69261, 29037, 91370, 31452, 
           161506, 27949, 29236, 15855, 162541, 161900, 157800, 90014, 120276, 57310, 
           92875, 140990, 11693, 78368, 62286, 80503, 72673, 121791, 12663, 90678]

# Subtract 1 from each element
updated_numbers = [x - 1 for x in numbers]

# Print the result
print(updated_numbers)


[110223, 146929, 2940, 104544, 62326, 29759, 96890, 47025, 117732, 163206, 61450, 20589, 145422, 33882, 4523, 81254, 82143, 85138, 167093, 125903, 116417, 158097, 95232, 81835, 84070, 53849, 112263, 17237, 34055, 10707, 164818, 32412, 75064, 139893, 161232, 100687, 148029, 55852, 23076, 152166, 44377, 111682, 145090, 132166, 83408, 153864, 143857, 111282, 150656, 36239, 142479, 112753, 149468, 50327, 163800, 45303, 155755, 151189, 160714, 8875, 116826, 142383, 143573, 136795, 277, 120988, 120990, 151641, 44082, 94916, 32814, 117613, 98216, 41936, 16625, 163884, 69260, 29036, 91369, 31451, 161505, 27948, 29235, 15854, 162540, 161899, 157799, 90013, 120275, 57309, 92874, 140989, 11692, 78367, 62285, 80502, 72672, 121790, 12662, 90677]


In [2]:
import argparse

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

from logger import Logger


class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc


def main():
    parser = argparse.ArgumentParser(description='OGBN-Arxiv (GNN)')
    parser.add_argument('--device', type=int, default=0)
    parser.add_argument('--log_steps', type=int, default=1)
    parser.add_argument('--use_sage', action='store_true')
    parser.add_argument('--num_layers', type=int, default=3)
    parser.add_argument('--hidden_channels', type=int, default=256)
    parser.add_argument('--dropout', type=float, default=0.5)
    parser.add_argument('--lr', type=float, default=0.01)
    parser.add_argument('--epochs', type=int, default=500)
    parser.add_argument('--runs', type=int, default=10)
    args = parser.parse_args()
    print(args)

    device = f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)

    dataset = PygNodePropPredDataset(name='ogbn-arxiv',
                                     transform=T.ToSparseTensor())

    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    data = data.to(device)

    split_idx = dataset.get_idx_split()
    train_idx = split_idx['train'].to(device)

    if args.use_sage:
        model = SAGE(data.num_features, args.hidden_channels,
                     dataset.num_classes, args.num_layers,
                     args.dropout).to(device)
    else:
        model = GCN(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout).to(device)

    evaluator = Evaluator(name='ogbn-arxiv')
    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, split_idx, evaluator)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--device DEVICE] [--log_steps LOG_STEPS]
                             [--use_sage] [--num_layers NUM_LAYERS]
                             [--hidden_channels HIDDEN_CHANNELS]
                             [--dropout DROPOUT] [--lr LR] [--epochs EPOCHS]
                             [--runs RUNS]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/kriti-arora/.local/share/jupyter/runtime/kernel-70624918-e461-4d71-a8c0-cb5f48e503ea.json


SystemExit: 2

/home/kriti-arora/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
